In [53]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
import numpy as np

In [59]:

energy_consumption = pd.read_csv("energy_consumption.csv")
base_station_basic_info = pd.read_csv("./base_station_basic_info.csv")
cell_level_data = pd.read_csv("cell_level_data.csv")

In [79]:

energy_consumption.rename(columns = {'Time':'Date-Time'}, inplace = True)

##breaking down Date-Time field
energy_consumption = energy_consumption[energy_consumption["Date-Time"].apply(lambda x: len(x.split(" ")) == 2)]
energy_consumption["Time"] = energy_consumption["Date-Time"].apply(lambda x : x.split(" ")[1])
energy_consumption["Date"] = energy_consumption["Date-Time"].apply(lambda x : x.split(" ")[0])
energy_consumption["Month"] = energy_consumption["Date"].apply(lambda x: int(x.split("/")[0]))
energy_consumption["Day"] = energy_consumption["Date"].apply(lambda x: int(x.split("/")[1]))
energy_consumption = energy_consumption.drop(columns=["Date","Date-Time"])


In [80]:

##convert Time and Base Station to usable Values
energy_consumption["Time"] = energy_consumption["Time"].apply(lambda x : int(x.split(":")[0]))
energy_consumption["BS"] = energy_consumption["BS"].apply(lambda x : int(x.split("_")[1]))

In [60]:

cell_level_data.rename(columns = {'Time':'Date-Time'}, inplace = True)
##breaking down Date-Time field
cell_level_data = cell_level_data[cell_level_data["Date-Time"].apply(lambda x: len(x.split(" ")) == 2)]
cell_level_data["Time"] = cell_level_data["Date-Time"].apply(lambda x : x.split(" ")[1])
cell_level_data["Date"] = cell_level_data["Date-Time"].apply(lambda x : x.split(" ")[0])
cell_level_data["Month"] = cell_level_data["Date"].apply(lambda x: int(x.split("/")[0]))
cell_level_data["Day"] = cell_level_data["Date"].apply(lambda x: int(x.split("/")[1]))
cell_level_data = cell_level_data.drop(columns=["Date","Date-Time"])
cell_level_data.head()

,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,Time,Month,Day
0,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,1:00,1,1
1,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,2:00,1,1
2,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,3:00,1,1
3,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,4:00,1,1
4,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,5:00,1,1


In [61]:
cell_level_data = cell_level_data[cell_level_data["BS"].apply(lambda x: len(x.split("_")) == 2)]
cell_level_data["BS"] = cell_level_data["BS"].apply(lambda x : int(x.split("_")[1]))
cell_level_data["CellName"] = cell_level_data["CellName"].apply(lambda x : int(x.replace("Cell","")))
cell_level_data["Time"] = cell_level_data["Time"].apply(lambda x : int(x.split(":")[0]))
# cell_level_data[cell_level_data["BS"].apply(lambda x : "float" in str(type(x)))]

In [81]:
# B_0 = cell_level_data[cell_level_data.apply(lambda x: x["BS"] == 0 and x["CellName"] == 1,axis=1)]
B_0 = energy_consumption[energy_consumption.apply(lambda x: x["BS"] == 0,axis=1)]

,BS,Energy,Time,Month,Day
0,0,64.27503737,1,1,1
1,0,55.90433483,2,1,1
2,0,57.6980568,3,1,1
3,0,55.15695067,4,1,1
4,0,56.05381166,5,1,1
